In this notebook, I will check the input to chromopainter/finestructure, as well as create some supporting files. 

In [1]:
%run ../scripts/notebook_settings.py

In [2]:
meta_data_samples = pd.read_table("../data/metadata_with_x_missing.txt", sep=" ")
vcf = "/faststorage/project/baboondiversity/data/haploidified_chrX_males/hap_chrX_males.vcf.gz"

In [3]:
callset = allel.read_vcf(vcf)

In [4]:
individuals = callset["samples"]

In [5]:
individuals

array(['PD_0215', 'PD_0217', 'PD_0223', 'PD_0227', 'PD_0228', 'PD_0229',
       'PD_0230', 'PD_0231', 'PD_0233', 'PD_0234', 'PD_0235', 'PD_0236',
       'PD_0268', 'PD_0269', 'PD_0637', 'PD_0658', 'PD_0687', 'PD_0688',
       'PD_0689', 'PD_0690', 'PD_0691', 'PD_0746', 'PD_0747', 'PD_0748',
       'PD_0752', 'PD_0753', 'PD_0755', 'PD_0756', 'PD_0757', 'PD_0761',
       'PD_0762', 'PD_0763', 'PD_0764', 'PD_0766', 'PD_0767', 'PD_0768',
       'PD_0771', 'PD_0775', 'Sci_30877', 'PD_0201', 'PD_0202', 'PD_0204',
       'PD_0207', 'PD_0218', 'PD_0220', 'PD_0242', 'PD_0266', 'PD_0267',
       'PD_0493', 'PD_0494', 'PD_0496', 'PD_0497', 'PD_0498', 'PD_0502',
       'PD_0504', 'PD_0505', 'PD_0508', 'PD_0642', 'PD_0650', 'PD_0654',
       'PD_0679', 'PD_0681', 'PD_0684', 'PD_0720', 'PD_0722', 'PD_0723',
       'PD_0725', 'PD_0727', 'PD_0728', 'PD_0729', 'PD_0731', 'PD_0732',
       'PD_0733', 'PD_0734', 'PD_0735', 'PD_0739', 'PD_0740', 'PD_0741',
       'PD_0742', 'PD_0743', 'PD_0749', 'PD_0750'

In [34]:
meta_data_samples.loc[(meta_data_samples.PGDP_ID == "PD_0687") & (meta_data_samples.Sex == "M")]

,PGDP_ID,Provider_ID,Provider,Genus,Species,Origin,Sex,address,longitude,latitude,callset_index,C_origin,x_missing
119,PD_0687,130 / 12MKM1150715,Knauf/Chuma/Roos,Papio,cynocephalus,"Mikumi, Tanzania",M,"Mikumi, Kilosa, Morogoro, Coastal Zone, Tanzania",36.977032,-7.404852,119,"Cynocephalus, Central Tanzania",0.015864


In [13]:
meta_data_samples.loc[(meta_data_samples.Origin == "Ruaha, Tanzania") & (meta_data_samples.Sex == "M")]

,PGDP_ID,Provider_ID,Provider,Genus,Species,Origin,Sex,address,longitude,latitude,callset_index,C_origin,x_missing
26,PD_0215,13RUM1130716,Knauf/Chuma/Roos,Papio,cynocephalus,"Ruaha, Tanzania",M,"Ruaha, Ulanga, Morogoro, Coastal Zone, Tanzania",36.714814,-8.895386,26,"Cynocephalus, Central Tanzania",0.014343
28,PD_0217,17RUM5140716,Knauf/Chuma/Roos,Papio,cynocephalus,"Ruaha, Tanzania",M,"Ruaha, Ulanga, Morogoro, Coastal Zone, Tanzania",36.714814,-8.895386,28,"Cynocephalus, Central Tanzania",0.015386


In [16]:
meta_data_samples.C_origin.unique()

array(['Cynocephalus, Central Tanzania', 'Anubis, Kenya',
       'Kindae, Zambia', 'Gelada, Captive', 'Hamadryas, Ethiopia',
       'Anubis, Tanzania', 'Cynocephalus, Western Tanzania',
       'Papio, Senegal', 'Ursinus, Zambia', 'Anubis, Ethiopia'],
      dtype=object)

In [17]:
meta_data_samples.Origin.unique()

array(['Mikumi, Tanzania', 'Aberdare, Kenya', 'Chunga, Zambia', 'captive',
       'Awash, Ethiopia', 'Serengeti, Tanzania', 'Arusha, Tanzania',
       'Ngorongoro, Tanzania', 'Ruaha, Tanzania', 'Gombe, Tanzania',
       'Udzungwa, Tanzania', 'Mahale, Tanzania', 'Katavi, Tanzania',
       'Selous, Tanzania', 'Lake Manyara, Tanzania',
       'Tarangire, Tanzania', 'Niokolo-Koba, Senegal',
       'Issa Valley, Tanzania', 'Dendro Park, Zambia', 'Filoha, Ethiopia',
       'Gog Woreda, Gambella region, Ethiopia'], dtype=object)

In [54]:
pop_list = []
inclusion_list = []
for i in individuals:
    if i[0] == "S":
        i = i[4:]
        inclusion_list.append(0)
    else:
        if meta_data_samples.loc[meta_data_samples.PGDP_ID == i]["Species"].values[0] == "ursinus (grayfoot)":
            inclusion_list.append(0)
        elif meta_data_samples.loc[meta_data_samples.PGDP_ID == i]["Species"].values[0] == "papio":
            inclusion_list.append(0)
        else:
            inclusion_list.append(1)
    t_pop = meta_data_samples.loc[meta_data_samples.PGDP_ID == i]["Origin"].values[0]
    C_origin = meta_data_samples.loc[meta_data_samples.PGDP_ID == i]["C_origin"].values[0]
    if t_pop == 'Mikumi, Tanzania' and i != "PD_0687":
        t_pop = "Cynocephalus_Mikumi"
    elif t_pop == 'Serengeti, Tanzania' or t_pop == 'Gombe, Tanzania':
        t_pop = "Anubis_Gombe_Serengeti"
    elif C_origin ==  'Cynocephalus, Central Tanzania':
        t_pop = "Cynocephalus_Ruaha_Udzungwa_Selous"
    elif C_origin == 'Anubis, Tanzania':
        t_pop = "Anubis_Tarangire_Ngorongoro_Manyara"
    else:
        t_pop = C_origin.replace(" ", "_").replace(",", "" )
    pop_list.append(t_pop)

In [55]:
len(pop_list)

126

#Old Version
pop_list = []
inclusion_list = []
for i in individuals:
    if i[0] == "S":
        i = i[4:]
        inclusion_list.append(0)    
    else:
        inclusion_list.append(1)    
    if meta_data_samples.loc[meta_data_samples.PGDP_ID == i]["Species"].values[0] == "ursinus (grayfoot)":
        pop_list.append("Ursinus")
    else:
        t_pop = meta_data_samples.loc[meta_data_samples.PGDP_ID == i]["C_origin"].values[0]
        t_pop = t_pop.replace(" ", "_").replace(",", "" )
        pop_list.append(t_pop)

In [56]:
pop_df = pd.DataFrame()
pop_df["ID"] = individuals
pop_df["population"] = pop_list
pop_df["inclusion"] = inclusion_list

In [58]:
pop_df.to_csv("/faststorage/project/baboondiversity/data/haploidified_chrX_males/idfile.ids",
              sep=" ", header=False, index=False)

In [12]:
meta_data_samples_Sci = meta_data_samples.copy()
for i, row in meta_data_samples_Sci.iterrows():
    if row.PGDP_ID[0] != "P":
        meta_data_samples_Sci.at[i, "PGDP_ID"] = "Sci_"+str(row.PGDP_ID)

In [22]:
meta_data_samples_Sci.to_csv("../data/metadata_Sci_included.txt",
              sep=" ", header=False, index=False)
meta_data_samples_Sci.iloc[8:11]

,PGDP_ID,Provider_ID,Provider,Genus,Species,Origin,Sex,address,longitude,latitude,callset_index,C_origin,x_missing
8,Sci_97124,97124,Jolly/Phillips-Conroy,Papio,hamadryas,"Awash, Ethiopia",M,"አዋሽ / Awash, Zone 3, አፋር ክልል / Afar, ኢትዮጵያ",40.167674,8.992496,8,"Hamadryas, Ethiopia",0.021821
9,PD_0067,1043,Roos,Theropithecus,gelada,captive,M,"SDSU Captive Wildlife Research Facility, Brook...",-96.793280,44.334031,9,"Gelada, Captive",0.024995
10,PD_0199,09SNF1101115,Knauf/Chuma/Roos,Papio,anubis,"Serengeti, Tanzania",F,"Serengeti, Mara, Lake Zone, Tanzania",34.742544,-1.996626,10,"Anubis, Tanzania",0.003591


Generating an idfile with all individuals

In [3]:
zarr_dir = "/faststorage/project/baboondiversity/data/PG_panu3_zarr_12_03_2021/callset.zarr/chr20"
#Opening the zarr data
callset = zarr.open_group(zarr_dir, mode="r")

In [4]:
individuals = callset["samples"][:]

In [5]:
meta_data_samples_Sci = meta_data_samples.copy()
for i, row in meta_data_samples_Sci.iterrows():
    if row.PGDP_ID[0] != "P":
        meta_data_samples_Sci.at[i, "PGDP_ID"] = "Sci_"+str(row.PGDP_ID)

In [11]:
meta_data_samples_Sci[:9].PGDP_ID.values

array(['Sci_16066', 'Sci_16098', 'Sci_30877', 'Sci_30977', 'Sci_34449',
       'Sci_34474', 'Sci_38168', 'Sci_97074', 'Sci_97124'], dtype=object)

Converting ldhat files to genetic maps.
Assuming that effective population size for chrX is 25000.

In [3]:
ldhat_path = "/faststorage/project/primatediversity/data/PG_baboons_pananu3_23_2_2021_ldhat/maps/chrX.map"
uniform_rec_path = "/faststorage/project/baboondiversity/data/haploidified_chrX_males/uniform_rec.recombfile"
phase_file = "/faststorage/project/baboondiversity/data/haploidified_chrX_males/chrX_v2.phase"

In [4]:
ldhat_map = pd.read_csv(ldhat_path, sep="\t")
uni_map = pd.read_csv(uniform_rec_path, sep=" ")

In [5]:
f=open(phase_file)
lines=f.readlines()
pos_line = lines[2]
pos_list = [int(x) for x in pos_line.split(" ")[1:]]
f.close()

In [6]:
ldhat_map

,POS,Mean_rho,Median_rho,L95,U95
0,157,2.08310,2.07973,1.69130,2.45233
1,170,2.08542,2.08008,1.70064,2.44515
2,240,2.09169,2.08121,1.71609,2.44834
3,279,2.09265,2.08121,1.71767,2.44834
4,311,2.09261,2.08121,1.73304,2.44515
...,...,...,...,...,...
341637,143689426,0.55022,0.62866,0.02716,0.97126
341638,143691423,0.55087,0.62797,0.02716,0.97731
341639,143691435,0.55180,0.62794,0.02716,0.97868
341640,143691467,0.55218,0.62652,0.02716,0.97868


In [7]:
uni_map

,start.pos,recom.rate.perbp
0,37,1.000000e-07
1,57,1.000000e-07
2,157,1.000000e-07
3,170,1.000000e-07
4,216,1.000000e-07
...,...,...
2923207,143691509,1.000000e-07
2923208,143691526,1.000000e-07
2923209,143691548,1.000000e-07
2923210,143691549,1.000000e-07


Math: Ldhat estimates it as 4*Ne*r/kb, and I want it in M/b.
With an additional twist that ldhat does not have all positions present.
I set Ne as 10000, but it could be many other sizes.
This conversion is done by dividing by 4*Ne, and then also dividing by 1000 to go to bases.

In [8]:
ldhat_map["convert_r"] = ldhat_map.Mean_rho/(4*10000*1000)

Many positions are not present in the recombination map made by Juraj, so these spaces have to be filled.

In [9]:
ld_hat_i = 0
ldhat_pos = ldhat_map["POS"][0]
ld_hat_prev_pos = 0
last_ldhat = int(ldhat_map["POS"][-2:-1])
last_uni = int(uni_map["start.pos"][-1:])
recom_rate = []
next_pos = 0
for i, row in uni_map.iterrows():
    current_pos = row["start.pos"]
    if current_pos >= ldhat_pos and current_pos < last_ldhat:
        ld_hat_i += 1
        ld_hat_prev_pos = ldhat_pos
        ldhat_pos = ldhat_map["POS"][ld_hat_i]
    if last_uni == next_pos:
        recom_rate.append(0)
    else:
        next_pos = uni_map["start.pos"][i+1]
        recom_rate.append(ldhat_map["convert_r"][ld_hat_i]*(next_pos-current_pos)/(ldhat_pos-ld_hat_prev_pos))

In [10]:
uni_map["recom_variable"] = recom_rate

In [11]:
uni_map["recom_variable"].mean()

5.261494756899169e-10

In [12]:
uni_map.to_csv("/faststorage/project/baboondiversity/data/haploidified_chrX_males/approx_rec_all_pos.recombfile",
              sep=" ", index=False, columns=["start.pos", "recom_variable"])

Now I have the opposite problem - when removing sites with missing state, the recombination file is too big.
As always, it cannot just be subset, as you then begin missing some recombination.

In [13]:
len(uni_map), len(pos_list), len(pos_list)/len(uni_map)

(2923212, 2440624, 0.8349117340788147)

In [14]:
sum_recomb = []
current_i = 0
current_recomb = 0
last_pos = pos_list[-1]
for i, row in uni_map.loc[uni_map["start.pos"] >= pos_list[0]].iterrows():
    if pos_list[current_i+1] == last_pos:
        sum_recomb.append(current_recomb)
        sum_recomb.append(0)
        break
    if row["start.pos"] >= pos_list[current_i+1]:
        sum_recomb.append(current_recomb)
        current_recomb = 0
        current_i += 1
    current_recomb += row.recom_variable

In [15]:
sum_recomb[:6], pos_list[:7], uni_map.loc[uni_map["start.pos"] >= pos_list[0]][:15]

([7.47032142857143e-09,
  1.1508994999999998e-07,
  1.0649761363636362e-08,
  1.124141477272727e-08,
  5.6740924760497355e-08,
  4.1947264573991034e-09],
 [216, 226, 311, 329, 348, 513, 531],
     start.pos  recom.rate.perbp  recom_variable
 4         216      1.000000e-07    7.470321e-09
 5         226      1.000000e-07    1.045845e-08
 6         240      1.000000e-07    5.231625e-08
 7         279      1.000000e-07    1.634852e-09
 8         280      1.000000e-07    1.961822e-08
 9         292      1.000000e-07    3.106218e-08
 10        311      1.000000e-07    5.916534e-10
 11        312      1.000000e-07    1.005811e-08
 12        329      1.000000e-07    1.124141e-08
 13        348      1.000000e-07    3.017432e-08
 14        399      1.000000e-07    4.194726e-09
 15        417      1.000000e-07    9.321614e-09
 16        457      1.000000e-07    1.305026e-08
 17        513      1.000000e-07    4.194726e-09
 18        531      1.000000e-07    9.321614e-10)

In [16]:
len(pos_list), len(sum_recomb)

(2440624, 2440624)

In [19]:
sum(sum_recomb)

0.001537803003156008

In [21]:
sum_map = pd.DataFrame(data={"start.pos": pos_list, "recom_variable": sum_recomb})
sum_map["recom_variable"] = np.clip(sum_map["recom_variable"], a_max=None, a_min=1.000000e-14)

In [23]:
sum(sum_map["recom_variable"])

0.0015378038297943347

In [24]:
sum(uni_map["recom_variable"])

0.0015380464611304033

In [17]:
sum_map.to_csv("/faststorage/project/baboondiversity/data/haploidified_chrX_males/approx_rec_subset_pos.recombfile",
              sep=" ", index=False, columns=["start.pos", "recom_variable"])

A couple of checks of the uni_map and sum_map

In [ ]:
sns.histplot(data=sum_map, x="recom_variable")

In [ ]:
sns.histplot(data=uni_map, x="recom_variable")